In [ ]:
import numpy as np
from scipy import ndimage
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
points = gpd.read_file(r"E:\points.geojson")

In [ ]:
points["measurement"].describe()

In [ ]:
import rasterio

def write_raster(filename, origin_x, origin_y, res_x, res_y, height, width, array, epsg="EPSG:28992"):
    """"Function to write a numpy array to a GeoTiff.""""
    
    transform = rasterio.transform.from_origin(origin_x, origin_y, res_x, res_y)
    with rasterio.open(filename, 'w', driver='GTiff', height=height,
                       width=width, count=1, dtype="float64",
                       crs=proj, transform=transform, nodata=0) as dst:
        dst.write(array, 1)


In [ ]:
from datetime import datetime

def heatmap(d, bins=(100,100), smoothing=1.3, cmap='jet', col=None, func=np.max, plot=True, log=False):
    """Function to create a heatmap from a geodataframe of points. The default
    behaviour returns a heatmap of the counts of points per cell, but by 
    providing a dataframe column and an aggragetion function it can can create a 
    heatmap using maximum, median or any other numpy function to aggregate multipel
    values in a bin. """
    
    # Helper functions to retrieve point coordinates
    def getx(pt):
        return pt.coords[0][0]

    def gety(pt):
        return pt.coords[0][1]
    
    # Helper function to bin points into the predefined number of bins 
    def bin_points(measurement):
        # Get the measurement index 
        m_i = measurement.i
        
        # In the matrix x and y are reversed
        m_x = gety(measurement.geometry)
        m_y = getx(measurement.geometry)
        
        # Calculate the coordinates in the matrix
        x_i = int(np.trunc((m_x - min_x) / resolution_x)) #- 1
        y_i = int(np.trunc((m_y - min_y) / resolution_y)) #- 1 
        
        return (x_i, y_i, measurement[col])
    
    # Helper function to output a heatmap using matplotlib's imshow
    def output_heatmap(heatmap, extent, na_value=None):
        # Log transform the values
        if log:
            heatmap = np.log(heatmap)
        
        # Replace negative infinite values with 0
        heatmap[np.isneginf(heatmap)] = 0
        
        # Plot the heatmap
        heatmap = ndimage.filters.gaussian_filter(heatmap, smoothing, mode='nearest')
        
        ## fix this part still: filter out nan values from actual zeros
        #if not na_value == None:
        #    logheatmap[np.abs(logheatmap - np.log(na_value)) < (np.max(logheatmap) - np.min(logheatmap))/100] = 0
       
        if plot:
            p = plt.imshow(heatmap, cmap=cmap, extent=extent)
            plt.gca().invert_yaxis()
            plt.colorbar(p, fraction=0.02, pad=0.04)
            return p
        else:
            heatmap = np.flip(heatmap, 0)
            filebasename = "{}_heatmap_{}".format(datetime.now().strftime("%Y%m%d%H%M"),
                                                     func.__name__.replace(".","_"))
            if log:
                filebasename += "_log"
            ext = "tiff"
            write_raster("{}.{}".format(filebasename,ext),
                         extent[0],
                         extent[2],
                         resolution_y,
                         resolution_x,
                         bins[0],
                         bins[1],
                         heatmap)

    # Create a 2d grid to bin the points. It stores the point count per bin. 
    x = list(d.geometry.apply(getx))
    y = list(d.geometry.apply(gety))
    heatmap_counts, xedges, yedges = np.histogram2d(y, x, bins=bins)
    extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]
    
    # If no column is provided to aggregate points per cell with return the 
    # heatmap with count values
    if col == None:
        return output_heatmap(heatmap_counts, extent)
    
    # If a column is defined, continue by calculating the bin dimensions
    # This will be used to perform a groupby operation for each bin coordinate. 
    min_x = np.min(xedges)
    min_y = np.min(yedges)
    resolution_x = (np.max(xedges) - min_x) / (bins[0] - 1)
    resolution_y = (np.max(yedges) - min_y) / (bins[1] - 1)
    
    # Fill a dataframe with the bin coordinates (x,y) and
    # the value to be aggregated (w)
    binned = d.apply(axis=1, func=bin_points).tolist()
    df = pd.DataFrame(binned, columns=["x","y","w"])  
    df_aggregated = df.groupby(["x","y"]).apply(func)
    
    # Remove the old x,y columns if they still exists, 
    # They will be overwritten by the groupby index.
    if "x" in df_aggregated:
        df_aggregated = df_aggregated.drop(["x"], axis=1)
    if "y" in df_aggregated:
         df_aggregated = df_aggregated.drop(["y"], axis=1)
            
    # Reset the groupby index (x,y) and restore the original column names
    df_aggregated = df_aggregated.reset_index()
    df_aggregated.columns = ["x","y","w"]

    # Create a 2d numpy array to store the aggregated values
    heatmap = np.empty(bins)
    na = 0 #np.max(df_aggregated.w)*2
    heatmap[:] = na
    for i,m in df_aggregated.iterrows():
        heatmap[int(m["x"]), int(m["y"])] = m["w"]
    
    # Return the heatmap with aggregated values
    return output_heatmap(heatmap, extent, na)
    

In [ ]:
heatmap(aero, bins=(450,450), cmap="BuPu", col="measurement", func=np.median, plot=False)